Code to generate Qwen-2 finetuning dataset

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#video_path = '/content/drive/MyDrive/SoccerNet/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_224p.mkv'
#label_path = '/content/drive/MyDrive/SoccerNet_labels/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/Labels-v2.json'

# video_path = '/content/drive/MyDrive/SoccerNet/europe_uefa-champions-league/2016-2017/2016-09-13 - 21-45 Barcelona 7 - 0 Celtic/1_224p.mkv'
# label_path = '/content/drive/MyDrive/eecs545/SoccerNet_labels/europe_uefa-champions-league/2016-2017/2016-09-13 - 21-45 Barcelona 7 - 0 Celtic/Labels-v2.json'

In [ ]:
# prompt: trim the video at video_path into 5 minute long chunks

# import moviepy.editor as mp
# import os

# def trim_video(video_path, output_dir, chunk_length=300): # 5 minutes = 300 seconds
#   """Trims a video into chunks of a specified length.

#   Args:
#     video_path: Path to the input video file.
#     output_dir: Directory to save the trimmed video chunks.
#     chunk_length: Length of each chunk in seconds.
#   """

#   if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

#   try:
#     video = mp.VideoFileClip(video_path)
#     duration = video.duration

#     for i in range(0, int(duration), chunk_length):
#       start_time = i
#       end_time = min(i + chunk_length, duration)

#       chunk = video.subclip(start_time, end_time)
#       output_path = os.path.join(output_dir, f"chunk_{i//chunk_length}.mp4")
#       chunk.write_videofile(output_path, codec="libx264", audio_codec="aac")
#       print(f"Created chunk: {output_path}")

#     video.close()

#   except Exception as e:
#     print(f"An error occurred: {e}")

# Example usage (assuming you have a directory to save the chunks)
# output_directory = "/content/drive/MyDrive/SoccerNet/europe_uefa-champions-league/2016-2017/2016-09-13 - 21-45 Barcelona 7 - 0 Celtic/1_224p_chunked"
# trim_video(video_path, output_directory)


  if event.key is 'enter':



Try creating a json entry for a sample query and answer

In [ ]:
# context_string = """
# Prompt: <video>\nWhat are all the match times that Goals occur?\n Answer: (2:15, 2:27), (3:40, 3:55), (4:10, 4:25)\n
# Prompt: <video>\nWhat are all the match times that Shots on target occur?\n Answer: (2:15, 2:27), (3:40, 3:55), (4:10, 4:25)\n
# Prompt: <video>\nWhat are all the match times that Corners occur?\n Answer: (0:10, 0:22), (4:20, 4:35)\n
# Prompt: <video>\nWhat are all the match times that Goals occur?\n Answer: (1:05, 1:18), (2:30, 2:45), (4:50, 4:59)\n
# Prompt: <video>\nWhat are all the match times that Corners occur?\n Answer: (2:46, 2:56), (4:34, 4:44)\n
# """

In [ ]:
context_string = """
Prompt: <video>\nHere is match commentary for a 5 min segment of a match <commentary>\n. Utilize the commentary and video clip of this segment to accurately find all the match times that Goals occur in this segment.\n Answer: (2:15, 2:27), (3:40, 3:55), (4:10, 4:25)\n
Prompt: <video>\nHere is match commentary for a 5 min segment of a match <commentary>\n. Utilize the commentary and video clip of this segment to accurately find all the match times that Shots on target occur in this segment.\n Answer: (2:15, 2:27), (3:40, 3:55), (4:10, 4:25)\n
Prompt: <video>\nHere is match commentary for a 5 min segment of a match <commentary>\n. Utilize the commentary and video clip of this segment to accurately find all the match times that Corners occur in this segment.\n Answer: (0:10, 0:22), (4:20, 4:35)\n
Prompt: <video>\nHere is match commentary for a 5 min segment of a match <commentary>\n. Utilize the commentary and video clip of this segment to accurately find all the match times that Goals occur in this segment.\n Answer: (1:05, 1:18), (2:30, 2:45), (4:50, 4:59)\n
Prompt: <video>\nHere is match commentary for a 5 min segment of a match <commentary>\n. Utilize the commentary and video clip of this segment to accurately find all the match times that Goals occur in this segment.\n Answer: (2:46, 2:56), (4:34, 4:44)\n
"""

In [ ]:
print(context_string)


Prompt: <video>
Here is match commentary for a 5 min segment of a match <commentary>
. Utilize the commentary and video clip of this segment to accurately find all the match times that Goals occur in this segment.
 Answer: (2:15, 2:27), (3:40, 3:55), (4:10, 4:25)

Prompt: <video>
Here is match commentary for a 5 min segment of a match <commentary>
. Utilize the commentary and video clip of this segment to accurately find all the match times that Shots on target occur in this segment.
 Answer: (2:15, 2:27), (3:40, 3:55), (4:10, 4:25)

Prompt: <video>
Here is match commentary for a 5 min segment of a match <commentary>
. Utilize the commentary and video clip of this segment to accurately find all the match times that Corners occur in this segment.
 Answer: (0:10, 0:22), (4:20, 4:35)

Prompt: <video>
Here is match commentary for a 5 min segment of a match <commentary>
. Utilize the commentary and video clip of this segment to accurately find all the match times that Goals occur in this s

In [ ]:
sample1 = {
    "id": "sample1",
    "video": "chunk_0.mp4",
    "conversations": [
      {
        "from": "human",
        "value": "<video>\nWhat are all the match times that goals occur?"
      },
      {
        "from": "gpt",
        "value": "02:44"
      }
    ]
  }

A sample can also contain multiple Q&A btw.

In [ ]:
key_events = set(["Corner", "Shots on target", "Goal", "Clearance", "Foul", "Indirect free-kick", "Direct free-kick", "Yellow card", "Red card", "Substitution"]) # simple key events to start
# "Indirect free-kick & Direct free-kick" = "free-kick"
# Foul & Yellow card & Red card = "Foul
free_kicks = set(["Indirect free-kick", "Direct free-kick"])
fouls = set(["Foul", "Yellow card", "Red card"])
# range_dict = {
#     "Corner": (-3, 9),  # The key moment is the kick and potential header/shot afterward.
#     "Shots on target": (-4, 8),  # Include buildup and immediate aftermath (goalkeeper save or rebound).
#     "Goal": (-7, 5),  # Capture buildup and celebration.
#     "Clearance": (-4, 8),  # Focus on the defensive action and transition.
#     "Foul": (-5, 7),  # Show the context and aftermath (e.g., protests, referee decision).
#     "Indirect free-kick": (-6, 6),  # Include positioning, execution, and potential follow-up play.
#     "Direct free-kick": (-7, 5),  # Cover the setup, execution, and potential goal/save.
#     "Yellow card": (-6, 6),  # Show what led to the booking and reaction.
#     "Red card": (-8, 4),  # More context needed, including referee consultation and player reaction.
#     "Substitution": (-5, 7),  # Capture substitution preparation and immediate impact.
# }

In [ ]:
def get_range(event, time_str):
    """
    Take an event and a time string as input, like: "Goal", "42:59"
    Return a string which captures a range according to range_dict like: "(42:52, 43:04)"
    """
    range_dict = {
        "Corner": (-3, 9),
        "Shots on target": (-4, 8),
        "Goal": (-7, 5),
        "Clearance": (-4, 8),
        "Foul": (-5, 7),
        "Free-kick": (-6, 6),
        "Substitution": (-5, 7),
    }

    if event not in range_dict:
        raise ValueError(f"Unknown event: {event}")

    # Parse the input time string
    minutes, seconds = map(int, time_str.split(":"))

    # Get the time range
    start_offset, end_offset = range_dict[event]
    start_time = (minutes * 60 + seconds) + start_offset
    end_time = (minutes * 60 + seconds) + end_offset

    # Convert back to MM:SS format
    start_minutes, start_seconds = divmod(start_time, 60)
    end_minutes, end_seconds = divmod(end_time, 60)

    return f"({start_minutes:02}:{start_seconds:02}, {end_minutes:02}:{end_seconds:02})"

def get_modified_label(label):
  if label in free_kicks:
    return "Free-kick"
  elif label in fouls:
    return "Foul"
  else:
    return label

I am thinking we can write a function that goes through a Labels.json file, and creates a sample for each key event in the video chunk.

This means we need to know the bounds of the video chunk we care about.

Questions can be formulated as "What are all the match times that {key event} occurs"

In [ ]:
# system_message = "You are a Vision Language Model specialized in identifying an action in a soccer match from a fixed set of action classes " + str(list(key_events)) + " as it occurs in a given soccer video. Your task is to observe the input video carefully and respond to the prompt. Prompts will ask for the match time of an action. You are to respond with a series of intervals of time (start, end) in real match time of when this action occurs. The video contains broadcast footage from previous soccer games between players of two teams distinguished by their team uniform. Focus on delivering accurate timestamps based on the live game time displayed in each frame. Absolutely avoid additional explanation. Here are some example prompts and answers in the format you are expect to follow: " + context_string
system_message = "You are a Vision Language Model specialized in identifying an action in a soccer match from a fixed set of action classes " + str(list(key_events)) + "as it occurs in a given soccer video. Your task is to observe the input video and commentary carefully and respond to the prompt. Prompts will ask for the match time of an action. You are to respond with a series of intervals of time (start, end) in real match time of when this action occurs. The video contains broadcast footage from previous soccer games between players of two teams distinguished by their team uniform. The commentary will come as a list of comments in the format [start_time, end_time, 'comment'], and time will be in seconds. Focus on delivering accurate timestamps based on the live game time displayed in each frame. Absolutely avoid additional explanation. Here are some example prompts and answers in the format you are expect to follow: " + context_string

In [ ]:
# import json
# video_path = "/scratch/eecs545w25_class_root/eecs545w25_class/highlights/SoccerNet/europe_uefa-champions-league/2016-2017/2016-09-13 - 21-45 Barcelona 7 - 0 Celtic/1_chunk_0.mp4"
# label_path = "/scratch/eecs545w25_class_root/eecs545w25_class/highlights/SoccerNet/europe_uefa-champions-league/2016-2017/2016-09-13 - 21-45 Barcelona 7 - 0 Celtic/Labels-v2.json"
# video_path = "/scratch/eecs545w25_class_root/eecs545w25_class/highlights/SoccerNet/video.mp4"
# video_path = "/europe_uefa-champions-league/2016-2017/2016-09-13 - 21-45 Barcelona 7 - 0 Celtic/1_chunk_0.mp4"
# data_path = folder of videos
# label_path = json file

# padding_dictionary = {}

# generate samples just over a label json file!!!!!

# def generate_samples(video_path, label_path, key_events, chunk_len, half, buffer_time):
#   # figure out the match time bounds that pertain to the video
#   video_path_list = video_path.split("/")
#   video_file = video_path_list[-1] # file name
#   video_name = video_file.split(".")[0]
#   chunk_number = video_name.split("_")[2] # 0,1,2,..,8

#   # in seconds
#   # i.e. for chunk 2, the start is 600 and the end is 900
#   start = int(chunk_number) * chunk_len
#   end = (int(chunk_number) + 1) * chunk_len

#   # read the labels
#   with open(label_path, 'r') as file:
#     labels = json.load(file) # lables is one big nested dictionary

#   labels = labels['annotations'] # list of dictionary of events

#   sample_list = {}

#   # for each key event, build the conversation
#   sample_count = 0

#   # loop through the labels
#   for event in labels:

#     # check if we are in bounds
#     # convert game time to seconds
#     gametime = event['gameTime'].split('-')[1]
#     match_half = int(event['gameTime'].split('-')[0])
#     gametime = int(gametime.split(':')[0]) * 60 + int(gametime.split(':')[1])

#     if gametime < start or gametime > end or half != match_half:
#       continue

#     if event['label'] in key_events:
#       # Modify the Yellow card, Red card, and Indirect, Direct free-kicks labels to Foul and Free-kick respectively
#       event['label'] = get_modified_label(event['label'])
#       # check if this is the first occurence of this event or not
#       if event['label'] not in sample_list:
#         # if this is the first occurence of the event we need to create a new json object
#         sample_list[event['label']] = {
#             "id": video_path_list[-2] + '/' + video_file + '/' + str(sample_count), # i.e. match/chunk_x.mp4/0, we may need to update ID based on real path
#             "video": video_file,
#             "conversations": [
#                 {
#                     "from": "human",
#                     "value": system_message + "<video>\nWhat are all the match times that " + event['label'] +  " occur?" # ADD SYSTEM PROMPT - Added (PM)
#                 },
#                 {
#                     "from": "gpt",
#                     "value": get_range(event['label'], event['gameTime'].split('-')[1]) # i.e. "02:44" # ADD PADDING OF +- 5 SO TIME IS A TUPLE - Added (PM)
#                 }
#               ]
#             }
#         sample_count += 1
#       else:
#         # not the first occurence of this event so modify the existing sample for this event
#         sample_list[event['label']]['conversations'][1]["value"] += ',' + get_range(event['label'], event['gameTime'].split('-')[1]) # ADD PADDING OF +- SO TIME IS A TUPLE - Added (PM)

#   return sample_list



In [ ]:
from collections import defaultdict

def generate_samples_from_labels(label_path, key_events, chunk_len, comments_base_path):
  # NEED TO KNOW THE EXACT FORMAT OF VIDEO PATH FOR JSON
  # video_path is everything after Soccernet?/
  # video_path = england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/chunks/1_000.mkv
  # label_path = /content/drive/MyDrive/SoccerNet_labels/spain_laliga/2016-2017/2016-11-19 - 18-15 Barcelona 0 - 0 Malaga/Labels-v2.json

  # construct video path
  video_path_list = label_path.split("/")
  league, season, match_title = video_path_list[5:8]
  chunks_folder = 'elephant'

  # read the labels
  with open(label_path, 'r') as file:
    labels = json.load(file) # lables is one big nested dictionary

  labels = labels['annotations'] # list of dictionary of events

  sample_list = {} # {key: (key_event, half, chunk #), value: json training sample}
  # sample_list_counts = defaultdict(int) # {key: (key_event, half, chunk #), value: count of number of events} used for "first", "second", "third", etc

  # for each key event, build the conversation
  sample_count = 0

  # Take only matches which have commentary
  if not os.path.exists(comments_base_path + "1_asr.json") or not os.path.exists(comments_base_path + "2_asr.json"):
    print(f"No commentary found: {comments_base_path}")
    return

  print(comments_base_path)
  # commentary
  comments = {
              1: process_and_save_comments(comments_base_path + "1_asr.json"),
              2: process_and_save_comments(comments_base_path + "2_asr.json")
              }


  # loop through the labels
  for event in labels:

    # convert game time to seconds
    gametime = event['gameTime'].split('-')[1]
    match_half = int(event['gameTime'].split('-')[0])
    gametime = int(gametime.split(':')[0]) * 60 + int(gametime.split(':')[1])
    chunk_number = int(gametime / chunk_len)
    if int(gametime) % chunk_len == 0 and chunk_number != 0:
      chunk_number -= 1
    # print(len(comments[match_half]))
    chunked_comments = "Here is match commentary for a 1 min segment of a match " + str(comments[match_half][chunk_number + 1]) + "\n"
    # chunked_comments = "Here is match commentary for a 5 min segment of a match " + str(comments[match_half][chunk_number + 1]) + "\n"
    video_path = f"{league}/{season}/{match_title}/{chunks_folder}/{match_half}_00{chunk_number}.mkv" if chunk_number < 10 else f"{league}/{season}/{match_title}/{chunks_folder}/{match_half}_0{chunk_number}.mkv"



    if event['label'] in key_events:
      # Modify the Yellow card, Red card, and Indirect, Direct free-kicks labels to Foul and Free-kick respectively
      event_name = get_modified_label(event['label'])
      # print(event_name)
      key = (event_name, match_half, chunk_number)

      # check if this is the first occurence of this event or not
      if key not in sample_list:
        # sample_list_counts[key] += 1

        # if this is the first occurence of the event we need to create a new json object
        sample_list[key] = {
            "id": video_path + '/' + str(sample_count), # i.e. match/chunk_x.mp4/0, we may need to update ID based on real path
            "video": video_path, # we have to know put this path together????
            "conversations": [
                {
                    "from": "human",
                    #"value": system_message + "<video>\n" + chunked_comments + f"Utilize the video clip and commentary of this segment to accurately find all the match times that {event_name} occurs in this segment.\n" # ADD SYSTEM PROMPT - Added (PM)
                    "value": "<video>\n" + chunked_comments + f"Utilize the video clip and commentary of this segment to accurately find all the match times that {event_name} occurs in this segment."
                },
                {
                    "from": "gpt",
                    "value": f"A {event_name} occurs at {get_range(event_name, event['gameTime'].split('-')[1])}." # A Goal occurs at (start, end).
                    #"value": get_range(event_name, event['gameTime'].split('-')[1]) # i.e. "02:44" # ADD PADDING OF +- 5 SO TIME IS A TUPLE - Added (PM)
                }
              ]
            }
        sample_count += 1
      else:
        # sample_list_counts[key] += 1
        # not the first occurence of this event so modify the existing sample for this event
        #sample_list[key]['conversations'][1]["value"] += ',' + get_range(event_name, event['gameTime'].split('-')[1]) # ADD PADDING OF +- SO TIME IS A TUPLE - Added (PM)
        sample_list[key]['conversations'][1]["value"] += f"A {event_name} occurs at {get_range(event_name, event['gameTime'].split('-')[1])}." # Goal 1 occurs at (start, end).

  return sample_list


# Here is the relevant match commentary: xxx. What are are all

In [ ]:
# video_path = "/scratch/eecs545w25_class_root/eecs545w25_class/highlights/SoccerNet/europe_uefa-champions-league/2016-2017/2016-09-13 - 21-45 Barcelona 7 - 0 Celtic/1_chunk_0.mp4"
# label_path = "/scratch/eecs545w25_class_root/eecs545w25_class/highlights/SoccerNet/europe_uefa-champions-league/2016-2017/2016-09-13 - 21-45 Barcelona 7 - 0 Celtic/Labels-v2.json"

In [ ]:
# chunk_0_samples = generate_samples(video_path, label_path, key_events, 300, 1) # -10 second + 3/4 seconds (valdiate the bounds), or do consistent length
# we could do a dictionary with something like key: event, value: +- window
# Modify prompt to contain in-context learning, from the human with some example Q&A

# chunk_0_samples

{'Corner': {'id': '2016-09-13 - 21-45 Barcelona 7 - 0 Celtic/chunk_0.mp4/0',
  'video': 'chunk_0.mp4',
  'conversations': [{'from': 'human',
    'value': '<video>\nWhat are all the match times that Corner occur?'},
   {'from': 'gpt', 'value': ' 02:31'}]},
 'Shots on target': {'id': '2016-09-13 - 21-45 Barcelona 7 - 0 Celtic/chunk_0.mp4/1',
  'video': 'chunk_0.mp4',
  'conversations': [{'from': 'human',
    'value': '<video>\nWhat are all the match times that Shots on target occur?'},
   {'from': 'gpt', 'value': ' 02:44'}]},
 'Goal': {'id': '2016-09-13 - 21-45 Barcelona 7 - 0 Celtic/chunk_0.mp4/2',
  'video': 'chunk_0.mp4',
  'conversations': [{'from': 'human',
    'value': '<video>\nWhat are all the match times that Goal occur?'},
   {'from': 'gpt', 'value': ' 02:44'}]}}

Code to extract paths and run generate_samples(), then save back to Drive

In [ ]:
base_video_path = "/content/drive/MyDrive/SoccerNet"
base_label_path = "/content/drive/MyDrive/SoccerNet_labels"

In [ ]:
from os import listdir
from os.path import isfile, join
import json
import os

# Iterate only through training leagues

training_samples = []

leagues = listdir(base_label_path)
for league in leagues:
  league_path = join(base_video_path, league)
  for season in listdir(league_path):
    season_path = join(league_path, season)
    for match in listdir(season_path):
      match_path = join(season_path, match)
      label_path = match_path + "/Labels-v2.json"
      comments_base_path = "/content/sn-echoes/Dataset/whisper_v2_en/" + league + "/" + season + "/" + match + "/"
      sample_list = generate_samples_from_labels(label_path, key_events, chunk_len=60, comments_base_path=comments_base_path)
      if sample_list is not None:
        training_samples += (sample_list.values())

json.dump(training_samples, open(join(base_video_path, "training_samples_minutes.json"), "w"))

/content/sn-echoes/Dataset/whisper_v2_en/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/
/content/sn-echoes/Dataset/whisper_v2_en/england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal/
No commentary found: /content/sn-echoes/Dataset/whisper_v2_en/england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United/
No commentary found: /content/sn-echoes/Dataset/whisper_v2_en/england_epl/2014-2015/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool/
/content/sn-echoes/Dataset/whisper_v2_en/england_epl/2014-2015/2015-04-11 - 19-30 Burnley 0 - 1 Arsenal/
/content/sn-echoes/Dataset/whisper_v2_en/england_epl/2014-2015/2015-05-17 - 18-00 Manchester United 1 - 1 Arsenal/
No commentary found: /content/sn-echoes/Dataset/whisper_v2_en/england_epl/2015-2016/2015-08-08 - 19-30 Chelsea 2 - 2 Swansea/
No commentary found: /content/sn-echoes/Dataset/whisper_v2_en/england_epl/2015-2016/2015-08-29 - 17-00 Chelsea 1 - 2 Crystal Palace/
No commentary found: /content/sn-ech

In [ ]:
training_samples[0]

{'id': 'england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/elephant/1_003.mkv/0',
 'video': 'england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/elephant/1_003.mkv',
 'conversations': [{'from': 'human',
   'value': '<video>\nHere is match commentary for a 1 min segment of a match [[\'03:00\', \'03:03\', \'It will be Trippier, the one in charge of putting the ball into play.\'], [\'03:03\', \'03:06\', \'There goes that delivery, closed, Courtois manages to clear it.\'], [\'03:07\', \'03:12\', \'Courtois, who is coming off a truly outstanding performance in the Champions League.\'], [\'03:12\', \'03:15\', \'Today there was perhaps speculation about the presence of Piter Cech as well, but\'], [\'03:15\', \'03:18\', "Mourinho has preferred to keep the Belgian\'s goal."], [\'03:18\', \'03:23\', \'Yes, because it is true that Courtois came from a slightly weaker performance.\'], [\'03:23\', \'03:28\', \'He had made some mistakes, especially in aerial balls, which is no

In [ ]:
import json
import random

# Load your dataset
with open(base_video_path + "/training_samples_minutes.json", 'r') as f:
    data = json.load(f)

random.seed(545)

# Shuffle the data for randomness
random.shuffle(data)

# reduced_size = 0.0625 # take 6.25 percent of data
# data = data[:int(len(data) * reduced_size)]

# Split into train and test
split_ratio = 0.8
split_idx = int(len(data) * split_ratio)
train_data = data[:split_idx]
test_data = data[split_idx:]

# Save them to new files
json.dump(train_data, open(join(base_video_path, "train_data_1_min.json"), "w"))
# step_size = len(train_data)//10
# for i in range(10):
  # json.dump(train_data[i*step_size: (i+1)*step_size], open(join(base_video_path, f"train_data_{i}_with_min.json"), "w"))
json.dump(test_data, open(join(base_video_path, "test_data_1_min.json"), "w"))

In [ ]:
!du -sh /content/drive/MyDrive/SoccerNet/test_data.json

8.7M	/content/drive/MyDrive/SoccerNet/test_data.json


In [ ]:
training_samples[0]

{'id': 'england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/elephant/1_003.mkv/0',
 'video': 'england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/elephant/1_003.mkv',
 'conversations': [{'from': 'human',
   'value': '<video>\nHere is match commentary for a 1 min segment of a match [[180.0, 183.4, \'It will be Trippier, the one in charge of putting the ball into play.\'], [183.70000000000002, 186.60000000000002, \'There goes that delivery, closed, Courtois manages to clear it.\'], [187.60000000000002, 192.4, \'Courtois, who is coming off a truly outstanding performance in the Champions League.\'], [192.4, 195.3, \'Today there was perhaps speculation about the presence of Piter Cech as well, but\'], [195.8, 198.60000000000002, "Mourinho has preferred to keep the Belgian\'s goal."], [198.70000000000002, 203.60000000000002, \'Yes, because it is true that Courtois came from a slightly weaker performance.\'], [203.9, 208.9, \'He had made some mistakes, especially in aer

In [ ]:
# from concurrent.futures import ProcessPoolExecutor

# def process_video(video_path):
#     """Helper function to process a single video."""
#     match_path, video = video_path
#     output_directory = join(match_path, f"{video.split('_')[0]}_chunked")
#     trim_video(join(match_path, video), output_directory)

# # Collect all video paths first
# video_paths = []
# for league in leagues:
#     league_path = join(base_video_path, league)
#     for season in listdir(league_path):
#         season_path = join(league_path, season)
#         for match in listdir(season_path):
#             match_path = join(season_path, match)
#             for video in listdir(match_path):
#                 video_paths.append((match_path, video))

# print(f"Found {len(video_paths)} videos to process.")

# # Run parallel processing
# with ProcessPoolExecutor() as executor:
#     executor.map(process_video, video_paths)

Found 493 videos to process.


Processing SoccerNet Echoes

In [ ]:
!git clone https://github.com/SoccerNet/sn-echoes.git

Cloning into 'sn-echoes'...
remote: Enumerating objects: 13999, done.
remote: Counting objects: 100% (2665/2665), done.
remote: Compressing objects: 100% (2652/2652), done.
remote: Total 13999 (delta 16), reused 2658 (delta 12), pack-reused 11334 (from 1)
Receiving objects: 100% (13999/13999), 148.54 MiB | 16.39 MiB/s, done.
Resolving deltas: 100% (4586/4586), done.
Updating files: 100% (4754/4754), done.


In [ ]:
model = "whisper_v2_en"
dataset_path = "/content/sn-echoes/Dataset/" + model

In [ ]:
from os import listdir
from os.path import isfile, join
import json

def process_and_save_comments(comments_path, chunk_len=60):
  # chunk comments into segments of given length
  comments = None
  with open(comments_path, 'r') as file:
        comments = json.load(file)
  # print(comments)
  total_segments = len(comments["segments"])
  chunked_comments = {}
  current_seg = 0
  for i in range(1, 100): # upper bound on number of chunks per half
    per_chunk_comments = []
    chunk_num = i
    merged_comment = ""
    if current_seg >= total_segments:
      break
    # 0th index is comment starting time, 1st index is comment ending time, 2nd index is comment
    start_t, end_t = comments["segments"][str(current_seg)][0], comments["segments"][str(current_seg)][1]
    # 0th index is comment starting time, 1st index is comment ending time, 2nd index is comment
    while current_seg < total_segments and comments["segments"][str(current_seg)][1] < chunk_num * chunk_len:
      # merged_comment += " " + comments["segments"][str(current_seg)][2]
      # end_t = max(end_t, comments["segments"][str(current_seg)][1])
      per_chunk_comments.append(comments["segments"][str(current_seg)])
      current_seg += 1
      while current_seg < total_segments and str(current_seg) not in comments["segments"]:
          # print(f'{current_seg} skipped')
          current_seg += 1
    # chunked_comments[chunk_num] = (start_t, end_t, merged_comment)
    for comment in per_chunk_comments:
      # convert start_t and end_t to time in minutes
      comment[0] = str(int(comment[0] // 60)).zfill(2) + ":" + str(int(comment[0]) % 60).zfill(2)
      comment[1] = str(int(comment[1] // 60)).zfill(2) + ":" + str(int(comment[1]) % 60).zfill(2)


    chunked_comments[chunk_num] = per_chunk_comments
  return chunked_comments

# half_path = "/content/sn-echoes/Dataset/whisper_v2_en/france_ligue-1/2015-2016/2015-09-26 - 18-30 Nantes 1 - 4 Paris SG/2_asr.json"


# leagues = listdir(dataset_path)

# new_comments = {}
# paths = []

# for league in leagues:
#   league_path = join(dataset_path, league)
#   for season in listdir(league_path):
#     season_path = join(league_path, season)
#     for match in listdir(season_path):
#       match_path = join(season_path, match)
#       for i, half in enumerate(listdir(match_path)):
#         half_path = join(match_path, half)
#         paths.append(half_path)
#         new_comments[half_path] = process_and_save_comments(half_path)

In [ ]:
# number of videos (500) * number of chunks (5) = 2500
print(len(paths) == len(new_comments))
print(len(paths))

True
718


In [ ]:
path = "/content/sn-echoes/Dataset/whisper_v2_en/england_epl/2016-2017/2016-10-22 - 17-00 Arsenal 0 - 0 Middlesbrough/1_asr.json"
new_comments[path]

{1: [[0.0, 3.0, 'Mike Dean'],
  [30.0, 32.0, 'Mike Dean'],
  [60.0, 62.0, 'Mike Dean'],
  [90.0, 92.0, 'Mike Dean'],
  [120.0, 122.0, 'Mike Dean'],
  [122.0, 124.0, 'Mike Dean'],
  [124.0, 126.0, 'Mike Dean'],
  [126.0, 128.0, 'Mike Dean'],
  [128.0, 130.0, 'Mike Dean'],
  [130.0, 132.0, 'Mike Dean'],
  [132.0, 134.0, 'Mike Dean'],
  [134.0, 136.0, 'Mike Dean'],
  [136.0, 138.0, 'Mike Dean'],
  [138.0, 140.0, 'Mike Dean'],
  [140.0, 142.0, 'Mike Dean'],
  [142.0, 144.0, 'Mike Dean'],
  [144.0, 146.0, 'Mike Dean'],
  [146.0, 148.0, 'Mike Dean'],
  [148.0, 150.0, 'Mike Dean'],
  [150.0, 152.0, 'Mike Dean'],
  [152.0, 154.0, 'Mike Dean'],
  [154.0, 156.0, 'Mike Dean'],
  [156.0, 158.0, 'Mike Dean'],
  [158.0, 160.0, 'Mike Dean'],
  [160.0, 162.0, 'Mike Dean'],
  [162.0, 164.0, 'Mike Dean'],
  [164.0, 166.0, 'Mike Dean'],
  [166.0, 168.0, 'Mike Dean'],
  [168.0, 170.0, 'Mike Dean'],
  [170.0, 172.0, 'Mike Dean'],
  [172.0, 174.0, 'Mike Dean'],
  [174.0, 176.0, 'Mike Dean'],
  [176.0, 178.0

In [ ]:
from collections import defaultdict

def LLM_generate_samples_from_labels(label_path, key_events, chunk_len, comments_base_path):
  # NEED TO KNOW THE EXACT FORMAT OF VIDEO PATH FOR JSON
  # video_path is everything after Soccernet?/
  # video_path = england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/chunks/1_000.mkv
  # label_path = /content/drive/MyDrive/SoccerNet_labels/spain_laliga/2016-2017/2016-11-19 - 18-15 Barcelona 0 - 0 Malaga/Labels-v2.json

  # construct video path
  video_path_list = label_path.split("/")
  league, season, match_title = video_path_list[5:8]
  chunks_folder = 'elephant'

  # read the labels
  with open(label_path, 'r') as file:
    labels = json.load(file) # lables is one big nested dictionary

  labels = labels['annotations'] # list of dictionary of events

  sample_list = {} # {key: (key_event, half, chunk #), value: json training sample}
  # sample_list_counts = defaultdict(int) # {key: (key_event, half, chunk #), value: count of number of events} used for "first", "second", "third", etc

  # for each key event, build the conversation
  sample_count = 0

  # Take only matches which have commentary
  if not os.path.exists(comments_base_path + "1_asr.json") or not os.path.exists(comments_base_path + "2_asr.json"):
    print(f"No commentary found: {comments_base_path}")
    return

  print(comments_base_path)
  # commentary
  comments = {
              1: process_and_save_comments(comments_base_path + "1_asr.json"),
              2: process_and_save_comments(comments_base_path + "2_asr.json")
              }


  # loop through the labels
  for event in labels:

    # convert game time to seconds
    gametime = event['gameTime'].split('-')[1]
    match_half = int(event['gameTime'].split('-')[0])
    gametime = int(gametime.split(':')[0]) * 60 + int(gametime.split(':')[1])
    chunk_number = int(gametime / chunk_len)
    if int(gametime) % chunk_len == 0 and chunk_number != 0:
      chunk_number -= 1
    # print(len(comments[match_half]))
    chunked_comments = str(comments[match_half][chunk_number + 1])
    # chunked_comments = "Here is match commentary for a 5 min segment of a match " + str(comments[match_half][chunk_number + 1]) + "\n"
    # video_path = f"{league}/{season}/{match_title}/{chunks_folder}/{match_half}_00{chunk_number}.mkv" if chunk_number < 10 else f"{league}/{season}/{match_title}/{chunks_folder}/{match_half}_0{chunk_number}.mkv"



    if event['label'] in key_events:
      # Modify the Yellow card, Red card, and Indirect, Direct free-kicks labels to Foul and Free-kick respectively
      event_name = get_modified_label(event['label'])
      # print(event_name)
      key = (event_name, match_half, chunk_number)

      # check if this is the first occurence of this event or not
      if key not in sample_list:
        # sample_list_counts[key] += 1

        # if this is the first occurence of the event we need to create a new json object
        sample_list[key] = {
            # "id": video_path + '/' + str(sample_count), # i.e. match/chunk_x.mp4/0, we may need to update ID based on real path
            # "video": video_path, # we have to know put this path together????
            "comments": chunked_comments,
            "timestamps": [get_range(event_name, event['gameTime'].split('-')[1])],
            "event": event_name
            # "conversations": [
            #     {
            #         "from": "human",
            #         #"value": system_message + "<video>\n" + chunked_comments + f"Utilize the video clip and commentary of this segment to accurately find all the match times that {event_name} occurs in this segment.\n" # ADD SYSTEM PROMPT - Added (PM)
            #         "value": "<video>\n" + chunked_comments + f"Utilize the video clip and commentary of this segment to accurately find all the match times that {event_name} occurs in this segment."
            #     },
            #     {
            #         "from": "gpt",
            #         "value": f"A {event_name} occurs at {get_range(event_name, event['gameTime'].split('-')[1])}." # A Goal occurs at (start, end).
            #         #"value": get_range(event_name, event['gameTime'].split('-')[1]) # i.e. "02:44" # ADD PADDING OF +- 5 SO TIME IS A TUPLE - Added (PM)
            #     }
            #   ]
            }
        sample_count += 1
      else:
        # sample_list_counts[key] += 1
        # not the first occurence of this event so modify the existing sample for this event
        #sample_list[key]['conversations'][1]["value"] += ',' + get_range(event_name, event['gameTime'].split('-')[1]) # ADD PADDING OF +- SO TIME IS A TUPLE - Added (PM)
        sample_list[key]['timestamps'].append(get_range(event_name, event['gameTime'].split('-')[1]))

  return sample_list


# Here is the relevant match commentary: xxx. What are are all

In [ ]:
from os import listdir
from os.path import isfile, join
import json
import os

# Iterate only through training leagues

LLM_training_samples = []

leagues = listdir(base_label_path)
for league in leagues:
  league_path = join(base_video_path, league)
  for season in listdir(league_path):
    season_path = join(league_path, season)
    for match in listdir(season_path):
      match_path = join(season_path, match)
      label_path = match_path + "/Labels-v2.json"
      comments_base_path = "/content/sn-echoes/Dataset/whisper_v2_en/" + league + "/" + season + "/" + match + "/"
      sample_list = LLM_generate_samples_from_labels(label_path, key_events, chunk_len=60, comments_base_path=comments_base_path)
      if sample_list is not None:
        LLM_training_samples += (sample_list.values())

json.dump(LLM_training_samples, open(join(base_video_path, "LLM_training_samples.json"), "w"))

/content/sn-echoes/Dataset/whisper_v2_en/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/
/content/sn-echoes/Dataset/whisper_v2_en/england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal/
No commentary found: /content/sn-echoes/Dataset/whisper_v2_en/england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United/
No commentary found: /content/sn-echoes/Dataset/whisper_v2_en/england_epl/2014-2015/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool/
/content/sn-echoes/Dataset/whisper_v2_en/england_epl/2014-2015/2015-04-11 - 19-30 Burnley 0 - 1 Arsenal/
/content/sn-echoes/Dataset/whisper_v2_en/england_epl/2014-2015/2015-05-17 - 18-00 Manchester United 1 - 1 Arsenal/
No commentary found: /content/sn-echoes/Dataset/whisper_v2_en/england_epl/2015-2016/2015-08-08 - 19-30 Chelsea 2 - 2 Swansea/
No commentary found: /content/sn-echoes/Dataset/whisper_v2_en/england_epl/2015-2016/2015-08-29 - 17-00 Chelsea 1 - 2 Crystal Palace/
No commentary found: /content/sn-ech

In [ ]:
print(len(LLM_training_samples))

28939


In [ ]:
LLM_training_samples[0]

{'comments': '[[180.0, 183.4, \'It will be Trippier, the one in charge of putting the ball into play.\'], [183.70000000000002, 186.60000000000002, \'There goes that delivery, closed, Courtois manages to clear it.\'], [187.60000000000002, 192.4, \'Courtois, who is coming off a truly outstanding performance in the Champions League.\'], [192.4, 195.3, \'Today there was perhaps speculation about the presence of Piter Cech as well, but\'], [195.8, 198.60000000000002, "Mourinho has preferred to keep the Belgian\'s goal."], [198.70000000000002, 203.60000000000002, \'Yes, because it is true that Courtois came from a slightly weaker performance.\'], [203.9, 208.9, \'He had made some mistakes, especially in aerial balls, which is not usual for him.\'], [209.2, 214.1, \'And I think Mourinho gave him a wake-up call the other day by leaving him as a substitute.\'], [215.6, 219.29999999999998, \'Although in reality I believe that Courtois feels like he is a starter, because he is a starter.\'], [219